In [3]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
import random
import sys
import os

# Load and preprocess the text dataset
path = tf.keras.utils.get_file('shakespeare.txt', origin='https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path, 'rb').read().decode(encoding='utf-8').lower()
print(f'Corpus length: {len(text)} characters')

# Create character mappings
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Prepare sequences
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print(f'Number of sequences: {len(sentences)}')

# One-hot encode the data
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Helper function to sample predictions with temperature
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Train and generate text
for epoch in range(1, 5):
    print(f'\nEpoch {epoch}')
    model.fit(x, y, batch_size=128, epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0]:
        print(f'\n----- Temperature: {temperature}')
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print(f'----- Generating with seed: "{sentence}"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Corpus length: 1115394 characters
Number of sequences: 371785


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Epoch 1
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 313s 107ms/step - loss: 2.6102

----- Temperature: 0.2
----- Generating with seed: "ir counsels and their cares, digest thin"
ir counsels and their cares, digest thing the mord the will the his the word the with me the mant the hard the mare the hard me the with the sond the hard the mare the for the have to he ward the mard the have the hard the hard the for the mand he mard the her the her the her the hing the his sore the his sor the with the buth me the beand the mere the come the hare the mart the hard the bere to mo to the come and come the hard the and 

----- Temperature: 0.5
----- Generating with seed: "ir counsels and their cares, digest thin"
ir counsels and their cares, digest thing ancenist to her buch stee to you be cone bor the the with hak to her donger mord wir ghe in the mars our beathe so no seer love som mor i gon wath then seand i come, i will whing him bold conte have the sirs in wirp and the covint manite her are her my to 